In [4]:
from keras import models
import utils
from keras import backend as K
from tensorflow import saved_model
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl
import tensorflow as tf
import shutil
import os

# Save the model.

In [2]:
print("Loading model...")

model_name_voxnet = utils.get_latest_model("voxnet")
model_name_voxnet = "20180804-0930-voxnet-model.h5"
print(model_name_voxnet)
model_voxnet = models.load_model(model_name_voxnet)
model_voxnet.summary()
print("")

print("Done.")

Loading model...
20180804-0930-voxnet-model.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 32, 32, 32, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 14, 14, 14, 32)    4032      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 12, 12, 32)    27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 6, 6, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6912)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               884864    
_________________________________________________________________
dense_2 (Dense)              

# Save the model.

In [5]:
export_path = model_name_voxnet.replace(".h5", "")
if os.path.exists(export_path):
    shutil.rmtree(export_path)

session = K.get_session()
graph = K.get_session().graph

prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def(
        {"image": model_voxnet.input}, 
        {"prediction": model_voxnet.output})

builder = saved_model_builder.SavedModelBuilder(export_path)
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
session.run(init_op)

builder.add_meta_graph_and_variables(
      session, [tag_constants.SERVING],
      signature_def_map={
           signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
               prediction_signature,
      },
      legacy_init_op=legacy_init_op)

builder.save()      

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: 20180804-0930-voxnet-model/saved_model.pb


b'20180804-0930-voxnet-model/saved_model.pb'